# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import datetime as dt
from collections import Counter
import requests
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from shapely.geometry import LineString, Point
from shapely import wkt
from dotenv import load_dotenv, find_dotenv
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.vector_layers import Circle, CircleMarker
import matplotlib
import ast

# Data

In [2]:
# import renatl info and drop irrelevant columns
bike_df_all = pd.read_csv('raw_data/bike_loans.csv', sep=';').drop(columns=['Importe', 
                                                                        'DescripcionImporte', 
                                                                        'operario'])
bike_df_all['Fecha_Prestamo'] = bike_df_all['Fecha_Prestamo'].astype('datetime64[ns]')
bike_df_all['Fecha_Devolucion'] = bike_df_all['Fecha_Devolucion'].astype('datetime64[ns]')

bike_df_all.dtypes

Id_Historico_Prestamo               int64
Id_Usuario                          int64
Id_Tag_Bicicleta                   object
Fecha_Prestamo             datetime64[ns]
Fecha_Devolucion           datetime64[ns]
Id_Aparcamiento_Origen              int64
Posicion_Origen                     int64
Id_Aparcamiento_Destino           float64
Posicion_Destino                  float64
Num_Bici_Hist                     float64
dtype: object

In [3]:
bike_df_all = bike_df_all.dropna()

# Locate individual bike at station

In [24]:
# filter to 2019 data => most representative year
bikes_19 = bike_df_all[bike_df_all.Fecha_Devolucion.dt.year == 2019].reset_index()
bikes_19['Id_Aparcamiento_Destino'] = bikes_19['Id_Aparcamiento_Destino'].astype(int)

In [25]:
# assign df a individual hour of the year per row
def get_hr(x):
    time = x
    hour = dt.datetime(time.year, time.month,time.day,time.hour)
    return hour
    
bikes_19.loc[:,'hour'] = bikes_19.Fecha_Devolucion.apply(get_hr)

In [26]:
bikes_19

,index,Id_Historico_Prestamo,Id_Usuario,Id_Tag_Bicicleta,Fecha_Prestamo,Fecha_Devolucion,Id_Aparcamiento_Origen,Posicion_Origen,Id_Aparcamiento_Destino,Posicion_Destino,Num_Bici_Hist,hour
0,440864,10948569,20272,00FB73222B5E,2019-02-20 14:08:04,2019-02-20 14:26:08,9,1,43,1.0,1366.0,2019-02-20 14:00:00
1,586995,12896369,135209,002772222B5E,2019-08-22 09:19:51,2019-08-22 09:20:31,44,15,44,15.0,976.0,2019-08-22 09:00:00
2,726893,12896264,26536,00094A72BF5B,2019-08-22 09:11:22,2019-08-22 09:20:53,9,1,70,1.0,2699.0,2019-08-22 09:00:00
3,1028721,10948678,124512,00233C222B5E,2019-02-20 14:15:54,2019-02-20 14:26:28,89,2,56,1.0,638.0,2019-02-20 14:00:00
4,2665185,10948590,31795,003491222B5E,2019-02-20 14:09:15,2019-02-20 14:26:31,44,16,56,23.0,1362.0,2019-02-20 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3780276,11767100,14454800,27944,0000F69A5A88,2019-12-31 13:58:55,2019-12-31 14:25:38,40,1,50,1.0,541.0,2019-12-31 14:00:00
3780277,11767101,14454778,108800,00029A222B5E,2019-12-31 13:52:48,2019-12-31 14:40:52,28,1,50,1.0,715.0,2019-12-31 14:00:00
3780278,11767102,14454793,98449,00001619CE0A,2019-12-31 13:55:47,2019-12-31 14:41:39,113,1,28,1.0,3497.0,2019-12-31 14:00:00
3780279,11767103,14454774,129665,000036885A88,2019-12-31 13:51:28,2019-12-31 16:27:12,39,15,41,15.0,534.0,2019-12-31 16:00:00


In [39]:
# make sure df is sorted chronologically
bikes_19 = bikes_19.sort_values(by='Fecha_Devolucion')

# list all the appearing bikes and stations per individual hour of the year
bikes_19_by_bike = bikes_19.copy().groupby('hour').agg({'Id_Tag_Bicicleta':lambda x: list(x), 
                                                        'Id_Aparcamiento_Destino':lambda x: list(x),
                                                        'Id_Aparcamiento_Origen':lambda x: list(x)})

In [40]:
bikes_19_by_bike

,Id_Tag_Bicicleta,Id_Aparcamiento_Destino,Id_Aparcamiento_Origen
hour,,,
2019-01-02 05:00:00,"[008492222B5E, 008A71222B5E, 000A5E72BF5B, 006...","[9, 1, 9, 9, 21, 8, 2, 65, 57, 12, 54, 9, 10, ...","[7, 2, 9, 7, 7, 51, 1, 23, 21, 8, 56, 7, 8, 7,..."
2019-01-02 06:00:00,"[008A71222B5E, 007667222B5E, 008B66222B5E, 001...","[1, 34, 21, 8, 42, 63, 66, 66, 66, 66, 66, 66,...","[67, 10, 7, 12, 48, 12, 23, 23, 8, 57, 23, 39,..."
2019-01-02 07:00:00,"[002071222B5E, 009491222B5E, 00D383222B5E, 00E...","[4, 66, 66, 61, 67, 66, 8, 55, 19, 64, 64, 7, ...","[48, 11, 23, 1, 1, 45, 11, 3, 46, 56, 47, 62, ..."
2019-01-02 08:00:00,"[00E65F222B5E, 001D83222B5E, 008077222B5E, 007...","[43, 36, 8, 67, 7, 19, 66, 56, 20, 36, 67, 48,...","[2, 89, 20, 33, 67, 54, 49, 58, 4, 10, 41, 48,..."
2019-01-02 09:00:00,"[001C70222B5E, 00D676222B5E, 00EB73222B5E, 008...","[33, 45, 41, 62, 37, 57, 21, 8, 41, 5, 11, 2, ...","[63, 89, 67, 4, 44, 10, 12, 23, 24, 67, 8, 3, ..."
...,...,...,...
2019-12-31 12:00:00,"[009D72222B5E, 0000C612CE0A, 00C876222B5E, 000...","[40, 12, 22, 36, 118, 21, 99, 41, 21, 12, 51, ...","[11, 12, 49, 46, 108, 24, 21, 100, 21, 2, 51, ..."
2019-12-31 13:00:00,"[0000761ACE0A, 00001619CE0A, 000066EA5A88, 003...","[21, 113, 2, 89, 36, 62, 4, 2, 36, 43, 55, 95,...","[65, 105, 1, 9, 46, 7, 66, 97, 66, 66, 2, 45, ..."
2019-12-31 14:00:00,"[008B83222B5E, 00000611CE0A, 00797A222B5E, 00B...","[32, 97, 3, 55, 97, 41, 115, 97, 66, 21, 90, 5...","[32, 59, 58, 33, 40, 41, 104, 12, 66, 57, 6, 2..."


In [41]:
# only keep the _last_ entry of a bike per hour (dict will overwrite)
# we will count pick-up and drop-off in the SAME time, since the avg. renting duration is about 10 mins. 
bikes_19_by_bike['bikes_loc'] = None
for c, row in bikes_19_by_bike.head().iterrows():
#     print(c) # check progress
    dct = {}
    for i, bike in enumerate(row.Id_Tag_Bicicleta):
        dct[bike] = (row.Id_Aparcamiento_Destino[i], row.Id_Aparcamiento_Origen[i])
    bikes_19_by_bike.at[c, 'bikes_loc'] = dct

# move hour to columns and only leave the dict
bikes_19_by_bike = bikes_19_by_bike[['bikes_loc']].reset_index()

# count the appearance of each station per hour
bikes_19_by_bike['station_count'] = None
for c, row in bikes_19_by_bike.head().iterrows():
    stations = row.bikes_loc.values()
    orgs = [i[1] for i in stations]
    dests = [i[0] for i in stations]
    dct_org = dict(Counter(orgs))
    dct_dest = dict(Counter(dests))
    
    bikes_19_by_bike.at[c, 'station_count'] = dct_org, dct_dest
        

# make dict column visible
bikes_19_by_bike.head(50).style.set_properties(subset=['bikes_loc'], **{'width': '150px'})

In [ ]:
bikes_19_by_bike

In [7]:
bikes_19_by_bike.to_csv('raw_data/stations_count_hr.csv')

In [11]:
# get all bike ids numbers of last hour in 2018 and their locations
## filter by before 2019 and match it with all apearing bikes, take the .head(1) to have the location of the bike

# then take station count dict and update appearing stations